In [1]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 21.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors

In [ ]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

full = pd.concat([train, test], axis = 0).reset_index(drop=True)
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

In [ ]:
from rdkit import DataStructs

allDescrs = [getMolDescriptors(m) for m in full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

# SMILES로부터 분자 객체 생성
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

# 화합물의 특성 벡터 생성 (여기서는 Morgan fingerprint 사용)
full['Fingerprint'] = full['Molecule'].apply(lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024))

fps = list(full['Fingerprint'])
distance_matrix = []
for i in range(len(full)):
    similarities = []
    for j in range(len(full)):
        sim = DataStructs.TanimotoSimilarity(fps[i], fps[j])
        similarities.append(sim)
    distance_matrix.append(similarities)

# distance matrix를 데이터프레임으로 변환
distance_df = pd.DataFrame(distance_matrix, columns=full.index, index=full.index)

In [ ]:
Data = pd.concat([full, full_Descrs], axis=1).drop(columns=['AlogP', 'Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

train = Data.iloc[:3498].dropna(axis=0)
test = Data.iloc[3498:]

In [ ]:
#결측 확인
for col in train.columns:
    has_missing_values = train[col].isna().any()
    count =  train[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

In [ ]:
for col in test.columns:
    has_missing_values = test[col].isna().any()
    count =  test[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

In [ ]:
# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features = [column for column in train.columns if column not in non_features]
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train)):
    df_train = train.iloc[train_index]
    df_valid = train.iloc[valid_index]

    x_train = df_train[features].values
    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid = df_valid[features].values
    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    # Create and compile the model
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    # Create and compile another model for HLM
    model_hlm = Sequential()
    model_hlm.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(1024, activation='relu'))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(512, activation='relu'))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(256, activation='relu'))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(1))

    model_hlm.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[RootMeanSquaredError()])


    # Train the model
    checkpoint_mlm = ModelCheckpoint(f'model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model.fit(x_train, y_mlm_train, epochs=500, batch_size=32, verbose=1, validation_data=(x_valid, y_mlm_valid),
              callbacks=[checkpoint_mlm, early_stopping_mlm])

    reg_mlms.append(model)

    checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model_hlm.fit(x_train, y_hlm_train, epochs=500, batch_size=32, verbose=1, validation_data=(x_valid, y_hlm_valid),
                  callbacks=[checkpoint_hlm, early_stopping_hlm])

    reg_hlms.append(model_hlm)

    # Calculate RMSE for MLM predictions
    y_mlm_pred = model.predict(x_valid)
    mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
    print(mlm_rmse)
    mlm_rmse_scores.append(mlm_rmse)

    # Calculate RMSE for HLM predictions
    y_hlm_pred = model_hlm.predict(x_valid)
    hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    print(hlm_rmse)
    hlm_rmse_scores.append(hlm_rmse)

In [ ]:
from keras.models import load_model

# Load MLM models
mlm_models = [load_model(f'model_mlm_fold{i}.h5') for i in range(n_splits)]

# Load HLM models
hlm_models = [load_model(f'model_hlm_fold{i}.h5') for i in range(n_splits)]

x_test = test[features].values

mlm_predictions = []
hlm_predictions = []

for model_mlm, model_hlm in zip(mlm_models, hlm_models):
    mlm_predictions.append(model_mlm.predict(x_test))
    hlm_predictions.append(model_hlm.predict(x_test))

# Convert prediction lists to numpy arrays
mlm_predictions = np.array(mlm_predictions)
hlm_predictions = np.array(hlm_predictions)

mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

In [ ]:
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = mlm_ensemble_prediction
df_submission["HLM"] = hlm_ensemble_prediction
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [ ]:
# load data
df = test

# predict
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = np.mean([reg_mlm.predict(df[features].values) for reg_mlm in reg_mlms], axis = 0)
df_submission["HLM"] = np.mean([reg_hlm.predict(df[features].values) for reg_hlm in reg_hlms], axis = 0)
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")